# Trabalho de casa #1

**Data**: 11-10-2021

## Informações sobre o aluno

**Nome**: José Pedro Ribeiro Peixoto<br>
**Número**: PG47381<br>
**Curso**: Mestrado em Engenharia Informática<br>
 

## Enunciado 
### O "Clube Desportivo de Ribeirão" tem as seguintes regras:
 - Todos os sócios que usam bigode, são de Ribeirão e são casados.
 - Cada sócio do clube que não é de Ribeirão tem que usar camisola amarela.
 - Os sócios casados não podem assistir aos jogos ao Domingo.
 - Um sócio vai aos jogos ao Domingo se e só se é de Ribeirão.
 - Cada sócio usa bigode ou não usa camisola amarela.
 - Todos os sócios de Ribeirão usam bigode.

**1\.** Por forma a codificar este puzzle como problema _SAT_, defina um  conjunto adequadode variáveis proposicionais, exprima as regras acima como fórmulas proposicionais, e converta essas fórmulas para CNF.

### Variáveis proposicionais

- **A** : O sócio usa bigode
- **B** : O sócio é Ribeironense
- **C** : O sócio é casado
- **D** : O sócio usa camisola amarela
- **E** : O sócio assiste aos jogos ao Domingo

### Regras

Todos os sócios que usam bigode, são de Ribeirão e são casados.

`A -> (B ∧ C)`

---

Cada sócio do clube que não é de Ribeirão tem de usar camisola amarela

`¬B -> D`

---

Os sócios casados não podem assistir aos jogos ao Domingo.

`C -> ¬E`

---

Um sócio vai aos jogos ao Domingo se e só se é de Ribeirão.

`E <-> B`

---

Cada sócio usa bigode ou não usa camisola amarela.

`A ∨ ¬D`

--- 

Todos os sócios de Ribeirão usam bigode.

`B -> A`

### Converter para CNF

```
     (A -> (B ∧ C)) ∧ (¬B -> D) ∧ (C -> ¬E) ∧ (E <-> B) ∧ (A ∨ ¬D) ∧ (B -> A) 
<->  (¬A ∨ (B ∧ C)) ∧ (B ∨ D) ∧ (¬C ∨ ¬E) ∧ ((E -> B) ∧ (B -> E)) ∧ (A ∨ ¬D) ∧ (¬B ∨ A)
<-> (¬A ∨ B) ∧ (¬A ∨ C) ∧ (B ∨ D) ∧ (¬C ∨ ¬E) ∧ (¬E ∨ B) ∧ (¬B ∨ E) ∧ (A ∨ ¬D) ∧ (¬B ∨ A)
```


**2\.** Codifique o problema num _SAT_ solver e comprove que o conjunto de regras é consistente.

In [ ]:
!pip install python-sat[pblib,aiger]

In [5]:
from pysat.solvers import Minisat22

s = Minisat22()                # Cria o solver

'''
A : 1
B : 2
C : 3
D : 4
E : 5
'''

s.add_clause([-1, 2])          # ¬A ∨  B
s.add_clause([-1, 3])          # ¬A ∨  C
s.add_clause([ 2, 4])          #  B ∨  D
s.add_clause([-3,-5])          # ¬C ∨ ¬E
s.add_clause([-5, 2])          # ¬E ∨  B
s.add_clause([-2, 5])          # ¬B ∨  E
s.add_clause([-1,-4])          # A  ∨ ¬D
s.add_clause([-2, 1])          # ¬B ∨  A

if s.solve():                  # testa a satisfatibilidade
    print("SAT")
    print(s.get_model())       # imprime o modelo
else: 
    print("UNSAT")

s.delete()                     # apaga o solver s

SAT
[-1, -2, -3, 4, -5]


Existe um conjunto de atirbuições (às variáveis proposicionais) que satisfaz o conjunto de regras:

```
SAT
[-1, -2, -3, 4, -5]
```
I.e. 

**A** : Falso<br>
**B** : Falso<br>
**C** : Falso<br>
**D** : Verdadeiro<br>
**E** : Falso<br>

Logo, o conjunto de regras é consistente (tal como é informado pelo _PySAT_ `SAT`).


**3\.** Use agora o SAT solver para o ajudar a responder às seguintes questões: 

 **(a)** - A afirmação "Quem usa bigode não pode ir ao jogo ao Domingo." é correta?

 ---

 ```   
       A -> ¬E
  <-> ¬A ∨ ¬E
 ```

 Acrescenta-se esta clausula ao código da pergunta **2.** e sse o resultado for `SAT` então a afirmação é correta. 

In [6]:
from pysat.solvers import Minisat22

s = Minisat22()                # Cria o solver

'''
A : 1
B : 2
C : 3
D : 4
E : 5
'''

s.add_clause([-1, 2])          # ¬A ∨  B
s.add_clause([-1, 3])          # ¬A ∨  C
s.add_clause([ 2, 4])          #  B ∨  D
s.add_clause([-3,-5])          # ¬C ∨ ¬E
s.add_clause([-5, 2])          # ¬E ∨  B
s.add_clause([-2, 5])          # ¬B ∨  E
s.add_clause([-1,-4])          # A  ∨ ¬D
s.add_clause([-2, 1])          # ¬B ∨  A

# Nova clausula
s.add_clause([-1, -5])          # ¬A ∨ ¬E

if s.solve():                  # testa a satisfatibilidade
    print("SAT")
    print(s.get_model())       # imprime o modelo
else: 
    print("UNSAT")

s.delete()                     # apaga o solver s

SAT
[-1, -2, -3, 4, -5]


```
SAT
[-1, -2, -3, 4, -5]
```

Logo, é correta. (Já se podia esperar este resultado, dado que a nova clausula era `-1 ∨ -5`. Algo que o modelo obtido anteriormente já comprovava)

 **(b)** - Pode um membro de camisola amarela ser casado?

 ---

 ```
 D ∧ C
 ```
Acrescenta-se estas duas clausulas ao código da pergunta 2. e sse o resultado for SAT então a afirmação é correta. 


In [7]:
from pysat.solvers import Minisat22

s = Minisat22()                # Cria o solver

'''
A : 1
B : 2
C : 3
D : 4
E : 5
'''

s.add_clause([-1, 2])          # ¬A ∨  B
s.add_clause([-1, 3])          # ¬A ∨  C
s.add_clause([ 2, 4])          #  B ∨  D
s.add_clause([-3,-5])          # ¬C ∨ ¬E
s.add_clause([-5, 2])          # ¬E ∨  B
s.add_clause([-2, 5])          # ¬B ∨  E
s.add_clause([-1,-4])          # A  ∨ ¬D
s.add_clause([-2, 1])          # ¬B ∨  A

# Novas clausulas

s.add_clause([4])          # D
s.add_clause([3])          # C

if s.solve():                  # testa a satisfatibilidade
    print("SAT")
    print(s.get_model())       # imprime o modelo
else: 
    print("UNSAT")

s.delete()                     # apaga o solver s

SAT
[-1, -2, 3, 4, -5]


```
SAT
[-1, -2, 3, 4, -5]
```

Logo, é correta.


 **(c)** - A afirmação "Afinal o clube não pode ter sócios Ribeironenses." é correta?

 ---

 ```
 ¬B
 ```
Acrescenta-se esta clausula ao código da pergunta 2. e sse o resultado for SAT então a afirmação é correta. 

In [8]:
from pysat.solvers import Minisat22

s = Minisat22()                # Cria o solver

'''
A : 1
B : 2
C : 3
D : 4
E : 5
'''

s.add_clause([-1, 2])          # ¬A ∨  B
s.add_clause([-1, 3])          # ¬A ∨  C
s.add_clause([ 2, 4])          #  B ∨  D
s.add_clause([-3,-5])          # ¬C ∨ ¬E
s.add_clause([-5, 2])          # ¬E ∨  B
s.add_clause([-2, 5])          # ¬B ∨  E
s.add_clause([-1,-4])          # A  ∨ ¬D
s.add_clause([-2, 1])          # ¬B ∨  A

# Nova clausula

s.add_clause([-2])          # ¬B

if s.solve():                  # testa a satisfatibilidade
    print("SAT")
    print(s.get_model())       # imprime o modelo
else: 
    print("UNSAT")

s.delete()                     # apaga o solver s

SAT
[-1, -2, -3, 4, -5]


```
SAT
[-1, -2, -3, 4, -5]
```
Logo, é correta.